# 🎬 Lights, Camera, ReAction Functions

This notebook builds a full **sitcom script generation pipeline** powered by **ReAct-based agents** — a powerful framework that combines **reasoning** (thinking through a problem) with **acting** (taking structured steps).

We start by **generating a sitcom concept** from creative keywords, then **outlining** the pilot episode scene-by-scene.  
**Scene 1** is generated directly from the outline to establish the world and tone.  
After that, each new scene is **scripted, reviewed, and improved** using specialized **ReAct agents** — a **Character Agent**, **Comedy Agent**, and **Environment Agent** — that simulate a real sitcom writers' room.

---

✅ **Benefits of using ReAct agents**:

- **More structured and transparent thinking:** Agents reason step-by-step before making edits.
- **Dynamic adaptation:** Agents flexibly plan the next creative moves based on scene history.
- **Better long-term coherence:** Scenes evolve logically, with tracked character growth, running jokes, emotional arcs, and worldbuilding.

---

After generation, each scene is **summarized and stored in a vector database**, enabling fast retrieval of scene metadata for future story planning.  
By combining **structured agent workflows** and **retrieval-augmented memory**, we bring sitcom worlds to life — one coherent, character-driven scene at a time.


In [1]:
#!pip install datasets
#!pip install sentence-transformers faiss-cpu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

import sys
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils") #Change to your path


In [4]:
!ls "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils"

character_agent.py     environment_helpers.py  script_review.py
character_helpers.py   outline_generation.py   text_utils.py
comedy_agent.py        __pycache__	       vector_db_utils.py
comedy_helpers.py      scene_planner_agent.py
environment_agents.py  screen_writing.py


In [5]:
from character_agent import CharacterAgent
from comedy_agent import ComedicAgent
from environment_agents import EnvironmentReActAgent
from scene_planner_agent import ScenePlannerAgent

from text_utils import (
    display_markdown_output,
    extract_scene,
    extract_title,
)

from outline_generation import (
    generate_sitcom_pitch,
    generate_pilot_episode_outline
)


from script_review import (
    validate_episode_outline,
)

from screen_writing import (
    generate_scene_1_script,
    generate_scene
)


from vector_db_utils import (
    summarize_scene,
    add_scene_to_vector_db
)

In [8]:
import numpy as np
import openai
from openai import OpenAI
from IPython.display import Markdown, display
import re
import time
from datetime import timedelta

In [9]:
print(openai.__version__)

1.75.0


In [10]:
from google.colab import userdata
import os

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

### Vector Database

In [11]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Vector store initialization
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

# To track metadata
vector_metadata = []


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Generating Pitch

- Generates pitch with key words

In [12]:
keywords = [
    "urban locksmith shop",
    "ex-con protagonist",
    "estranged daughter",
    "quirky parole officer",
    "second chances",
    "buddy comedy",
    "New York City",
    "heartfelt absurdism",
    "odd couple dynamic"
]

In [13]:
start_time = time.time()

# Generating Sitcom Pitch
sitcom_pitch = generate_sitcom_pitch(client, keywords)
display(Markdown(f"### Sitcom Concept:\n\n{sitcom_pitch}"))

end_time = time.time()
elapsed_time = end_time - start_time

print(" ")
print(f"Generation time: {elapsed_time:.2f} seconds")


### Sitcom Concept:

Title: "Unlocked"

Description: "Unlocked" is a heartwarming, absurdly comedic take on second chances set against the backdrop of a vibrant, urban locksmith shop located deep in the heart of New York City. The sitcom follows the life of our ex-con protagonist, Charlie, a former safe-cracker who's trying to straighten his life out. He opens a locksmith shop as a form of repentance and attempts to reconnect with his estranged daughter, Abby, who's wary of his criminal past but reluctantly joins him in his new venture. The show thrives on the odd couple dynamic of Charlie and his quirky parole officer, Marv, who surprisingly becomes Charlie's best friend and silent partner in the locksmith shop. Marv's unorthodox methods of parole supervision often lead to hilariously unpredictable situations. Laden with buddy comedy moments, "Unlocked" tackles the theme of redemption in a bustling city that never sleeps, all while showcasing the funny and often poignant trials of a father and daughter trying to rebuild their relationship amidst keys, locks, and the constant possibility of relapse.

 
Generation time: 12.57 seconds


### Episode Outline

In [14]:
outline = generate_pilot_episode_outline(client, sitcom_pitch)
display(Markdown(f"### Sitcom Concept:\n\n{outline}"))

### Sitcom Concept:

Episode Concept: In the pilot episode of "Unlocked", ex-con Charlie opens his locksmith shop, gets the surprise of his life when his daughter Abby shows up, and navigates the unexpected challenges of running a business under the watchful eye of his eccentric parole officer Marv.

Scene 1: "The Shop Opens" Charlie, with a mix of excitement and nervousness, opens his locksmith shop for the first time; a comedic exchange with a suspicious passerby sets the tone for his new life.

Scene 2: "Routine Check" Marv pops in for an unexpected parole check, leading to humorous misunderstandings about his unorthodox approach to parole supervision.

Scene 3: "Surprise Visit" Abby enters the shop, surprising Charlie and sparking a tense but ultimately heartwarming father-daughter reunion.

Scene 4: "Skeletons in the Closet" Charlie and Abby have a heartfelt conversation about their past, with Charlie expressing his hope for a second chance at being a better father.

Scene 5: "The First Customer" The shop's first customer brings a complex lock problem that is hilariously solved by Charlie's ex-con skills.

Scene 6: "Marv's Motives" Marv's frequent visits to the shop are revealed to be due to his interest in becoming a silent partner, leading to a comedic negotiation scene.

Scene 7: "Abby's Reluctance" Abby expresses her skepticism about Charlie's new venture and her fear of his possible relapse, adding depth to her character.

Scene 8: "The Odd Couple" Marv and Charlie's opposing personalities lead to a series of funny interactions as they try to run the shop together.

Scene 9: "Lock-Picking Lesson" Charlie gives Abby a lock-picking lesson, creating a heartfelt bonding moment between father and daughter.

Scene 10: "Marv's Unorthodox Methods" Marv's unconventional parole methods lead to a hilarious situation involving a potential customer and a misunderstanding about a stolen lock.

Scene 11: "Old Habits Die Hard" Charlie is tempted to return to his criminal ways, creating tension and underscoring his struggle for redemption.

Scene 12: "Abby's Decision" Abby decides to help her father, shifting the dynamic and setting the stage for future episodes.

Scene 13: "Partnership Problems" Charlie and Marv's partnership is tested by a ridiculous business situation, showcasing their comedic chemistry.

Scene 14: "Tricky Customer" A picky customer’s demands lead to humorous interactions and creative problem-solving by the team.

Scene 15: "Father-Daughter Moment" Abby opens up to Charlie about her feelings, and they share a touching father-daughter moment.

Scene 16: "Marv's Surprise" Marv surprises the duo with a witty idea to attract more customers, leading to a funny planning session.

Scene 17: "The Heist Temptation" A customer's valuable lock tempts Charlie, testing his resolve and creating a suspenseful, comedic situation.

Scene 18: "Unexpected Success" The team is surprised by a sudden influx of customers, leading to a series of hilarious mishaps.

Scene 19: "Redemption Realized" Charlie resists the temptation to return to crime, reinforcing the theme of redemption and strengthening his bond with Abby.

Scene 20: "Locking Up" The closing scene features Charlie, Abby, and Marv locking up after a long day, sharing a laugh, and leaving viewers eager for the next episode’s adventures.

In [15]:
validation = validate_episode_outline(client, sitcom_pitch, outline)
display(Markdown(f"### Coherence Evaluation:\n\n{validation}"))

### Coherence Evaluation:

Coherence: Yes

Reasoning:
- The outline is consistent with the sitcom's premise. It introduces Charlie's new life as a locksmith, his relationship with his estranged daughter Abby, and his quirky dynamic with his parole officer, Marv.
- The sequence of events is logical and gradually builds up the characters and their relationships. For example, it starts with Charlie opening his shop, then Abby's unexpected arrival, and finally Marv's involvement in the business.
- The tone of the sitcom is maintained throughout the outline. The humorous interactions, heartfelt moments, and the underlying theme of redemption are all well-balanced.
- The outline effectively sets up potential storylines for future episodes, such as the evolution of Charlie and Abby's relationship, the odd couple dynamic between Charlie and Marv, and Charlie's struggle to resist his criminal past.

## Scene 1 Generation

In [16]:
scene_1_desc = extract_scene(outline, 1)
print(scene_1_desc)

Scene 1: "The Shop Opens" Charlie, with a mix of excitement and nervousness, opens his locksmith shop for the first time; a comedic exchange with a suspicious passerby sets the tone for his new life.


In [17]:
# Extracting title
sitcom_title = extract_title(sitcom_pitch)
print(sitcom_title)

Unlocked


In [18]:
scene_1_script = generate_scene_1_script(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_1_desc,
    rag_context=None,
    line_target=(55, 75)
)

In [19]:
print(scene_1_script)


INT. CHARLIE'S LOCKSMITH SHOP - DAY

CHARLIE, a middle-aged man with a tool belt around his waist, hangs an "Open" sign on the door of his quaint locksmith shop. He surveys his new domain with a mix of excitement and trepidation. 

CHARLIE
(to himself)
Here goes nothing. 

Just as he turns back, a SUSPICIOUS PASSERBY, a grizzled man with a shifty gaze, peeks through the window.

SUSPICIOUS PASSERBY
(grinning)
You the new locksmith in town, eh?

CHARLIE
(smiling nervously)
Sure am! Can I help you with something?

SUSPICIOUS PASSERBY
Just sizing up the competition. 

[LAUGH TRACK]

CHARLIE
(looking confused)
Competition? You're a locksmith too?

SUSPICIOUS PASSERBY
(leaning in)
Not exactly. I specialize in... let's call it... "unauthorized access".

[LAUGH TRACK]

CHARLIE
(laughs uncomfortably)
So... a burglar?

SUSPICIOUS PASSERBY
(shrugging)
I prefer the term "lock enthusiast."

[LAUGH TRACK]

CHARLIE
(sarcastically)
Well, it's always good to meet a fellow enthusiast.

SUSPICIOUS PASS

### Adding Metadata to Vector Database

In [20]:
scene_metadata = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_1_script
)



In [21]:
# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata,
    full_script=scene_1_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


In [22]:
print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 1 


Scene 1
Summary: In this scene, Charlie, a middle-aged man, opens his new locksmith shop with a mix of excitement and trepidation. He is soon approached by a Suspicious Passerby who introduces himself as a "lock enthusiast", implying that he is a burglar. The interaction is filled with humor as they discuss their shared interest in locks, albeit from different perspectives. The Suspicious Passerby leaves Charlie with a cryptic warning about every lock being a challenge he accepts. Charlie, left slightly bewildered, acknowledges that he's embarked on a new chapter in his life, preparing himself for the challenges and eccentric characters his new venture will bring.
Characters: ['Charlie', 'Suspicious Passerby']
Location: Charlie's Locksmith Shop
Recurring Joke: None
Emotional Tone: Humorous, Tense


## Scene 2 Generation

In [23]:
scene_2_desc = extract_scene(outline, 2)
print(scene_2_desc)

Scene 2: "Routine Check" Marv pops in for an unexpected parole check, leading to humorous misunderstandings about his unorthodox approach to parole supervision.


### Character Agent

In [24]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

# Outputs
print(f"Scene {2} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


Scene 2 — Consistency: ❌ Inconsistent

Explanation:
 1. Consistency Verdict: No
2. Short Explanation: The character Marv has not been introduced or described in the character profiles, so it's impossible to determine if his behavior in the planned scene is consistent with his established profile. Charlie's consistency cannot be determined without the actual dialogue or actions from the scene.
3. Specific Suggestions if inconsistencies exist: Provide a character profile for Marv, including his personality traits, speaking style and quirks, relationships with other characters, and emotional arc. Also, provide more details about the planned scene, including the dialogue and actions of the characters, to determine if they are behaving consistently with their established profiles.

Interaction Improvement Recommendations:
 Interaction Recommendations:
- [Suggestion 1] Marv, being a parole officer, could enter Charlie's shop with a stern demeanor, setting a serious tone. This can lead Charli

### Comedy Agent

In [25]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

Is Consistent: True
Analysis:
 1. Detected Tone: Absurdity
2. Consistency Verdict: Yes
3. Short Explanation: The comedic tone of absurdity is consistent with the previous scene. Both scenes use unusual characters (a "lock enthusiast" and an unorthodox parole officer) and unexpected situations to create humor.
4. Overuse Check: None
5. Specific Suggestions if inconsistencies or overuse exist: Not applicable.
Recommendations:
 Interaction Recommendations:
- [Suggestion 1] Marv, the parole officer, could have a strange fascination with house plants and often gets distracted by them during his visits. In this scene, he could mistake a common houseplant for a rare, illegal species and start questioning the parolee about it, creating a humorous misunderstanding.

- [Suggestion 2] Marv could use unconventional methods for parole check, like using a pair of socks to check for dust under the couch or asking the parolee to do 10 push-ups to ensure they're staying healthy. This creates a funny co

### Enviroment Agent



In [26]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=1  # or 2, 3, 5 — however many past scenes you want it to look at
)

In [27]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

In [28]:
print(environment_details_suggestions)

Environment Details Suggestions:
- [Suggestion 1] As Charlie enters Marv's home, he's immediately greeted by an old, squeaky hamster wheel that Marv has repurposed as a noise-making security system to alert him of any intruders.
- [Suggestion 2] A kitchen table cluttered with Marv's paperwork, including his parole documents, but also a 'Parolees for Dummies' book and a DIY home decor magazine, suggesting Marv's attempts to turn his life around.
- [Suggestion 3] A peculiar smell wafts through Marv's home, a mix of brewed coffee and burnt toast, Marv's unique, albeit unsuccessful, attempt at cooking breakfast. The burnt toast is humorously mistaken for a potential fire hazard violation by Charlie.


### Scene Planner Agent

In [29]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_2_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=2
)

In [30]:
print(scene_2_plan)

Scene Plan:
Character Goals:
- [Goal 1] Marv, the parole officer, enters Charlie's locksmith shop with a stern demeanor, leading Charlie to initially mistake him for a customer with a "tough guy" act. Charlie tries to lighten the mood with a locksmith joke which earns a deadpan response from Marv.
- [Goal 2] Just as Marv is leaving the shop, Charlie gives him a “special” lock as a parting gift, with a cryptic comment about its function, leaving Marv puzzled but also intrigued.

Comedic Goal:
- [Goal 1] Marv carries a giant, outdated camcorder to document his parole visits, struggling to operate it and often accidentally zooming in too close or recording himself instead of the parolee.

Environment Detail:
- [Detail] As Charlie enters Marv's home, he's immediately greeted by an old, squeaky hamster wheel that Marv has repurposed as a noise-making security system to alert him of any intruders.

Creative Suggestion:
- [Suggestion] In the next scene, Charlie could visit Marv's home to fix 

### Generating Scene 2 Script

In [31]:
scene_2_script = generate_scene(
    client=client,
    scene_plan=scene_2_plan,
    scene_number= 2
)

print(scene_2_script)

INT. CHARLIE'S LOCKSMITH SHOP - DAY

Marv, a gruff parole officer holding a giant, outdated camcorder, walks in. Charlie, a friendly locksmith, looks up from his workbench, surprised by Marv's stern demeanor.

CHARLIE
(cheerful)
Well, aren't you a bolt out of the blue! What can I do for you today?

Marv grunts, fumbling with the camcorder. He accidentally zooms in on his own face, squinting into the lens.

MARV
(gruff)
I'm here for a parole visit. And it's Marv, not "bolt."

Charlie chuckles, trying to lighten the mood.

CHARLIE
(laughing)
Sorry, Marv. A locksmith humor. How about this one - why don't we play safe and avoid any lock-y puns?

Marv gives a deadpan look, not amused.

MARV
(deadpan)
That was... quite the knee-slapper, Charlie.

Charlie hands Marv a “special” lock from the counter.

CHARLIE
(teasing)
Here, a parting gift. It's a lock with a few tricks up its sleeve.

Marv looks puzzled but intrigued. He exits the shop, still struggling with his camcorder.

CUT TO:

INT. MAR

### Adding Metadata to Vector Database

In [32]:
scene_metadata_2 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_2_script
)


In [33]:
# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_2,
    full_script=scene_2_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


In [34]:
print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 2 


Scene 1
Summary: In this scene, Charlie, a middle-aged man, opens his new locksmith shop with a mix of excitement and trepidation. He is soon approached by a Suspicious Passerby who introduces himself as a "lock enthusiast", implying that he is a burglar. The interaction is filled with humor as they discuss their shared interest in locks, albeit from different perspectives. The Suspicious Passerby leaves Charlie with a cryptic warning about every lock being a challenge he accepts. Charlie, left slightly bewildered, acknowledges that he's embarked on a new chapter in his life, preparing himself for the challenges and eccentric characters his new venture will bring.
Characters: ['Charlie', 'Suspicious Passerby']
Location: Charlie's Locksmith Shop
Recurring Joke: None
Emotional Tone: Humorous, Tense

Scene 2
Summary: In this scene, Marv, a gruff parole officer, visits Charlie, a friendly locksmith, at his shop for a parole visit. Marv struggles with 

## Scene 3 Generation

In [35]:
scene_3_desc = extract_scene(outline, 3)
print(scene_3_desc)

Scene 3: "Surprise Visit" Abby enters the shop, surprising Charlie and sparking a tense but ultimately heartwarming father-daughter reunion.


### Character Agent

In [36]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

# Outputs
print(f"Scene {3} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


Scene 3 — Consistency: ✅ Consistent

Explanation:
 1. Consistency Verdict (Yes/No)
The text does not provide enough information to determine consistency.

2. Short Explanation (max 5 lines)
The text does not provide enough information about Abby's personality traits, speaking style, and quirks to determine if her behavior in the planned scene is consistent with her established profile. Similarly, the text does not provide enough information about Charlie's reaction to Abby's surprise visit to determine if his behavior is consistent with his established profile.

3. Specific Suggestions if inconsistencies exist
To determine consistency, the text should provide more information about Abby's personality traits, speaking style, and quirks. It should also provide more information about Charlie's reaction to Abby's surprise visit, including his emotions, dialogue, and actions.

Interaction Improvement Recommendations:
 Interaction Recommendations:
- Suggestion 1: Charlie could initially mist

### Comedy Agent

In [37]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

Is Consistent: True
Analysis:
 1. Detected Tone: Heartwarming humor
2. Consistency Verdict: Yes
3. Short Explanation: The humor in this scene is consistent with the previous scenes. It maintains the light-hearted and humorous tone, but adds a touch of warmth and sentimentality due to the father-daughter reunion.
4. Overuse Check: None
5. Specific Suggestions if inconsistencies or overuse exist: N/A
Recommendations:
 Interaction Recommendations:
- [Suggestion 1]: As Abby enters the shop, she knocks over a stack of cans in the doorway, causing a loud crash. Charlie, without seeing who it is, scolds "Hey, watch it!" before realizing it's his daughter. It can be a running joke that Charlie always scolds people for knocking over the cans without checking who it is.
- [Suggestion 2]: Abby, knowing Charlie's love for trivia, can try to ease the tension by challenging him to a trivia contest. They exchange a few questions back and forth, with Charlie getting his facts hilariously wrong but ins

### Enviroment Agent


In [38]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=2  # or 2, 3, 5 — however many past scenes you want it to look at
)

In [39]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

In [40]:
print(environment_details_suggestions)

Environment Details Suggestions:
- [Suggestion 1] As Marv enters the shop, a little bell jingles above the door, attracting everyone's attention. The bell is broken and jingles non-stop until Marv awkwardly reaches up to stop it, giving a humorous start to the scene.
- [Suggestion 2] The shopkeeper, an elderly man with a quirky character, is trying to control a naughty parrot that keeps repeating random customer conversations, adding a light-hearted and funny element to the background.
- [Suggestion 3] The shop has a distinct smell of old books and lavender air freshener, which Marv comments on, saying it's an unusual but surprisingly pleasant combination.


### Scene Planner Agent

In [41]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_3_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=3
)

In [42]:
print(scene_3_plan)

Scene Plan:
Character Goals:
- [Goal 1]: To showcase Charlie's humor and naivety, he initially mistakes Abby for a customer and greets her with his usual locksmith humor. Abby's response is warm but sardonic, hinting at their familiar relationship before she reveals her identity.
- [Goal 2]: To create an element of conflict, Abby expresses her concern about Charlie's new venture, citing the Suspicious Passerby incident as an example. Charlie responds by light-heartedly assuring her with a locksmith metaphor, showing his innovative thinking and also his tendency to handle tense situations with humor.

Comedic Goal:
- [Goal 1]: To lighten the mood, Abby challenges Charlie to a trivia contest. They exchange a few questions, with Charlie getting his facts hilariously wrong but insisting he's right until Abby proves him wrong with her smartphone.

Environment Detail:
- [Detail]: A little bell jingles above the door as Abby enters the shop. The bell is broken and jingles non-stop until Abby 

### Generating Scene 3 Script

In [43]:
scene_3_script = generate_scene(
    client=client,
    scene_plan=scene_3_plan,
    scene_number= 3
)

print(scene_3_script)

INT. CHARLIE'S LOCKSMITH SHOP - DAY

The little bell above the door JINGLES non-stop as ABBY, a witty woman in her late 30s enters. She awkwardly reaches up to stop it.

CHARLIE, a goofy locksmith in his early 40s, looks up from his work desk, his eyes sparkling with mischief.

CHARLIE
(cheerfully)
Hi there, ma'am! Got a lock that even Houdini couldn't crack?

Abby rolls her eyes, a grin creeping onto her face.

ABBY
(sarcastically)
Wow, Charlie. You've really outdone yourself with the humor today.

Charlie chuckles then realizing who it is, his face lights up.

CHARLIE
(surprised)
Abby! I didn't recognize you at first!

Abby crosses her arms, looking around the shop with concern.

ABBY
(worried)
Charlie, are you sure about this new venture? Remember that Suspicious Passerby incident?

Charlie waves off her concern, laughing it off.

CHARLIE
(teasing)
Ah, come on Abby! Life's like a tricky lock, sometimes it just takes a few wrong turns to find the right groove.

Abby smirks at Charlie

### Adding Metadata to Vector Database

In [44]:
scene_metadata_3 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_3_script
)


# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_3,
    full_script=scene_3_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 3 


Scene 1
Summary: In this scene, Charlie, a middle-aged man, opens his new locksmith shop with a mix of excitement and trepidation. He is soon approached by a Suspicious Passerby who introduces himself as a "lock enthusiast", implying that he is a burglar. The interaction is filled with humor as they discuss their shared interest in locks, albeit from different perspectives. The Suspicious Passerby leaves Charlie with a cryptic warning about every lock being a challenge he accepts. Charlie, left slightly bewildered, acknowledges that he's embarked on a new chapter in his life, preparing himself for the challenges and eccentric characters his new venture will bring.
Characters: ['Charlie', 'Suspicious Passerby']
Location: Charlie's Locksmith Shop
Recurring Joke: None
Emotional Tone: Humorous, Tense

Scene 2
Summary: In this scene, Marv, a gruff parole officer, visits Charlie, a friendly locksmith, at his shop for a parole visit. Marv struggles with 